<a href="https://colab.research.google.com/github/vineet-codes/defi-exploration/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
def get_usernames(r=2):
  inputstring = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0123456789'
  # print(len(inputstring))
  username_list =list(set([inputstring[x:y] for x, y in itertools.combinations(range(len(inputstring) + 1), r = r)]))
  return username_list

len(get_usernames())

2015

Questions: 

1. How does unstake work [user has to claim it ?]

In [3]:
class Vault:

  def __init__(self, name):
    # pool is of the form {user: [(amount_1, tick_1), (amount_2, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick is used for having a contcept of time in our analysis
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def balanceOf(self, user):
    user_txns = self.pool.get(user,[])
    if(len(user_txns) == 0):
      return 0 #f'It seems {user} has made no txns in to the Vault'
    return sum([amount for (amount, _) in user_txns])
    

  def shareOfUser(self, user):
    """
    return: users shares of the total pool
    """
    return self.balanceOf(user) / self.tvl()

  def deposit(self, amount, user):
    """
    return: tuple (user, updated balance)
    """
    if (user in self.pool.keys()):
      user_txns = self.pool.get(user, []) 
      user_txns.append((amount, self.tick))
      self.pool[user] = user_txns
    else:
      self.pool[user] = [(amount, self.tick)]
    return user, self.pool[user]

  def withdraw(self, withdraw_amount, user):
    """
    return: tuple (user, updated balance)
    """
    if(user not in self.pool.keys()):
      return f'{user} has not deposited in the vault'
    
    if(self.balanceOf(user) < withdraw_amount):
      return f'Not enough balance, {user} can one withdraw upto: {self.balanceOf(user)}'
    
    # db update
    # self.pool[user] = self.pool[user] - withdraw_amount
    self.pool[user].append((-1* withdraw_amount, self.tick))

    return user,self.balanceOf(user)

  def tvl(self):
    result = 0
    for user, txns in self.pool.items():
      user_balance = self.balanceOf(user)
      result += user_balance
    return result
      
    return sum([v for _,v in self.pool.items()])
  
  def get_pool(self):
    return self.pool



class Stakes:

  def __init__(self, name="YOP_LOCKBOX"):

    # pool is of the form {user: [(amount_1, duration_1, tick_1), (amount_2, duration_2, tick_2), (amount_1, duration_1, tick_2)]}
    self.pool = {}

    self.name = name
    # self.rewards_pool = rewards_pool
    # tick helps with time travel
    self.tick = 0

  def update_tick(self, step):
    self.tick += step

  def stake(self, user, amount, duration):
    """
    duratrion: in months
    """
    # assert self.tick == tick

    user_stakes = self.pool.get(user, [])
    # print(user_stakes)
    user_stakes.append((amount, duration, self.tick))
    self.pool[user] = user_stakes
    # self.rewards_pool.tick()
    return user, self.pool.get(user, [])

  def unstake(self):
    pass


  def get_pool(self):
    return self.pool

  def pool_size(self):
    return sum([amount*duration for user in self.pool.keys() for amount, duration, _ in self.pool[user]])

  def shareOfUser(self, user):
    totalPoolSize = self.pool_size()
    if(totalPoolSize == 0):
      return 0
    userPoolSize = sum([amount*duration for amount, duration, _ in self.pool.get(user, [(0,0,0)])])
    return userPoolSize / totalPoolSize



class Rewards:

  def __init__(self, vault, stakes, 
               weights=[0.2,0.8], 
               vault_stake_reward_split = [0.75, 0.25], 
               yearly_emissions = 3891931,
               yop_token_price = 1):
    
    self.tick = 0
    
    self.yearly_emissions = yearly_emissions

    self.vault = vault
    self.stakes = stakes

    self.weights = weights
    self.vault_stake_reward_split = vault_stake_reward_split
    self.yop_token_price = yop_token_price

    # list to store history state per tick
    self.reward_per_tick_history = []

  
  def update_tick(self, step):
    self.tick += step


  def calculate_rewards(self):

    result = {}
    
    total_power = 0
    for user in list(set(self.vault.pool.keys())):
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      total_power += vault_boost_multiplier * self.vault.balanceOf(user)

    # print(total_power)

    for user in list(set(self.vault.pool.keys())):
      # print("=========")
      vault_boost_multiplier = self.weights[0]*self.vault.shareOfUser(user) + self.weights[1]*self.stakes.shareOfUser(user)
      # print(f'{user} vault boost multiplier is: {vault_boost_multiplier}')
      # x = (vault_boost_multiplier * self.vault.balanceOf(user) / total_power) * ((self.vault_stake_reward_split[0]/4)*self.yearly_emissions) * (self.yop_token_price)
      x = ((vault_boost_multiplier * self.vault.balanceOf(user)) / total_power) * ((0.2)*self.yearly_emissions) * (self.yop_token_price)
      # print(f"{user} is getting {(x / self.vault.balanceOf(user))*100}% APY in the vault")
      result[user] = (x / self.vault.balanceOf(user))*100
    return result
 
    
  
  def tick_fn(self, step):


    # update clocks for everyone
    self.update_tick(step)
    self.vault.update_tick(step)
    self.stakes.update_tick(step)

    # TODO: calculate the rewards earned in this tick

    # TODO: unstake and calculate stake rewards to be persisted

    # calculate the current vault reward state to be persisted
    state = {}

    # print(f"===Tick: {self.tick}====")
    # print(self.calculate_rewards())
    # print("Vault pool",self.vault.pool)
    # print("Stake Pool", self.stakes.pool)

    state[f'apys_{self.tick}'] = self.calculate_rewards()
    state['vault_pool'] = self.vault.pool.copy()
    state['stake_pool'] = self.stakes.pool.copy()

    self.reward_per_tick_history.append(state)
    # print("========")

    
    
      

  


# v = Vault()
# print(v.tvl())
# print(v.balanceOf('dfg'))
# v.deposit(5000, 'e')
# print(v.tvl())

class User:

  def __init__(self, name):
    self.name = name

  def __repr__(self):
    return f'{self.name}'

  def vault_rewards(self):
    pass

  def stake_rewards(self):
    pass

In [4]:
usdc_vault = Vault('test')
yop_stakes = Stakes()
vineet = User("Vineet")
john = User("John")
anar = User("Anar")

reward_calculator = Rewards(usdc_vault, yop_stakes)

reward_calculator.tick_fn(1)

# users make deposit into vault
usdc_vault.deposit(50_000, vineet)
# yop_stakes.stake(john, 5000, 12)


reward_calculator.tick_fn(1)

usdc_vault.deposit(50_000, vineet)

reward_calculator.tick_fn(1)

usdc_vault.deposit(60_000, john)

# users stake some yop
yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)

yop_stakes.stake(vineet, 5000, 12)
reward_calculator.tick_fn(1)


reward_calculator.tick_fn(1)

In [5]:
usdc_vault.pool, usdc_vault.balanceOf(anar)

({John: [(60000, 3)], Vineet: [(50000, 1), (50000, 2)]}, 0)

In [6]:
usdc_vault.shareOfUser(vineet) + usdc_vault.shareOfUser(john) == 1

True

In [7]:
usdc_vault.withdraw(100000, vineet), usdc_vault.balanceOf(vineet), usdc_vault.tvl(), yop_stakes.get_pool()

((Vineet, 0), 0, 60000, {Vineet: [(5000, 12, 3), (5000, 12, 4)]})

In [8]:
def toy_sheet_setup(steps = 100):

  # create vault and stake pool
  usdc_vault = Vault('test vault')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  # create users
  vineet, john, anar = User("Vineet"), User("John"), User("Anar")

  # make some txns in vaults and stakes in some ticks
  # tick
  rewards.tick_fn(1)

  # users make deposit into vault
  usdc_vault.deposit(670_000, vineet)
  yop_stakes.stake(john, 100_000, 5)

  #tick 
  rewards.tick_fn(1)
  
  usdc_vault.deposit(2_000_000, john)

  # users stake some yop
  # yop_stakes.stake(vineet, 5000, 12)
  rewards.tick_fn(1)

  usdc_vault.deposit(5000, anar)

  yop_stakes.stake(anar, 50000, 5)
  rewards.tick_fn(1)

  # yop_stakes.stake(anar, 5000, 12)
  rewards.tick_fn(1)



  return rewards, usdc_vault, yop_stakes, [vineet, john, anar]

In [9]:
rewards, usdc_vault, yop_stakes, [vineet, john, anar] = toy_sheet_setup()

In [10]:
usdc_vault.pool, yop_stakes.pool, rewards.tick, rewards.calculate_rewards()

({Anar: [(5000, 3)], John: [(2000000, 2)], Vineet: [(670000, 1)]},
 {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
 5,
 {Anar: 14.840511248388019,
  John: 37.94960412560259,
  Vineet: 2.7838943172431097})

In [11]:
rewards.reward_per_tick_history

[{'apys_1': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_2': {Vineet: 116.1770447761194},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {Vineet: [(670000, 1)]}},
 {'apys_3': {John: 38.24237769141653, Vineet: 2.0206934584581293},
  'stake_pool': {John: [(100000, 5, 1)]},
  'vault_pool': {John: [(2000000, 2)], Vineet: [(670000, 1)]}},
 {'apys_4': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}},
 {'apys_5': {Anar: 14.840511248388019,
   John: 37.94960412560259,
   Vineet: 2.7838943172431097},
  'stake_pool': {Anar: [(50000, 5, 3)], John: [(100000, 5, 1)]},
  'vault_pool': {Anar: [(5000, 3)],
   John: [(2000000, 2)],
   Vineet: [(670000, 1)]}}]

In [12]:
common_users = list(set(rewards.vault.pool.keys()).intersection(set(rewards.stakes.pool.keys())))

In [13]:
common_users

[John, Anar]

## WIP: Setup Optimization simulation

In [14]:
import random

In [36]:
def setup_sim(steps=300):

  actions = ['deposit', 'withdraw', 'stake', 'unstake']

  # instantiate vaults and staking pool
  usdc_vault = Vault('USDC_GENESIS')
  yop_stakes = Stakes()
  rewards = Rewards(usdc_vault, yop_stakes)

  action_history = []

  # generate random users from population of all users
  usernames = get_usernames()
  user_population = [User(username) for username in usernames]

  for tick in range(1, steps+1):
    # update the tick
    rewards.tick_fn(1)
    # for each tick
    #  pick a random user and a random action he takes
    user = random.choice(user_population[:20])
    action_name = random.choice(actions)

    print(f"{user} {action_name}'s at tick {rewards.tick}")
    action_history.append({user: action_name})

    # execute the action
    if(action_name == 'deposit'):
      # deposit: user deposits a random amount in vault
      usdc_vault.deposit(10000, user)
    elif (action_name == 'withdraw'):
      balance = usdc_vault.balanceOf(user)
      usdc_vault.withdraw(balance/8, user)
    elif (action_name == 'stake'):
      yop_stakes.stake(user, 1000, 12)
    else:
      # unstake
      pass
  # print(rewards.calculate_rewards())
  return rewards, user_population, action_history

In [37]:
rewards, user_population, action_history = setup_sim()

KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz withdraw's at tick 1
VW withdraw's at tick 2
uvwx stake's at tick 3
qrstuvwxyz01 unstake's at tick 4
uvwx unstake's at tick 5
uvwxyz unstake's at tick 6
uvwxyz withdraw's at tick 7
uvwxyz unstake's at tick 8
MN deposit's at tick 9
JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0 stake's at tick 10
opq deposit's at tick 11
uvwxyz deposit's at tick 12
uvwxyz unstake's at tick 13
defghijklmnopqrstuvwxyz012 unstake's at tick 14
opq deposit's at tick 15
DEFGHIJKLMNOPQRSTUVWXYZabscdefgh unstake's at tick 16
opq unstake's at tick 17
uvwx unstake's at tick 18
TUVWXYZabscdefghijklmnopqrstuvwxyz0123 deposit's at tick 19
2345 unstake's at tick 20
MN unstake's at tick 21
WXYZabs unstake's at tick 22
ghijklmnopqrstuvwxyz012345 stake's at tick 23
2345 withdraw's at tick 24
uvwxyz withdraw's at tick 25
tuvwxyz0123456 unstake's at tick 26
ghijklmnopqrstuvwxyz012345 stake's at tick 27
defghijklmnopqrstuvwxyz012 unstake's at tick 28
gh withdraw's at tick 29
defgh

In [38]:
len(rewards.vault.pool.keys()), rewards.vault.tvl()

(20, 569447.4029541016)

In [39]:
rewards.tick, rewards.vault.tick, rewards.stakes.tick

(300, 300, 300)

In [40]:
rewards.calculate_rewards()

{2345: 152.31230442597732,
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: 167.03344085597195,
 HIJKL: 250.09638775331447,
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: 117.8027458427244,
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: 90.57113045738248,
 MN: 143.42749993270388,
 NOPQRSTUVWX: 64.50722052048394,
 PQRSTUVWXYZabscdef: 112.1477450029846,
 TUVWXYZabscdefghijklmnopqrstuvwxyz0123: 48.36648936525396,
 VW: 129.01444104096788,
 WXYZabs: 235.69088973247045,
 defghijklmnopqrstuvwxyz012: 87.36155781835012,
 gh: 266.7328998040833,
 ghijklmnopqrstuvwxyz012345: 146.01264317372778,
 jklmn: 170.3059070551305,
 opq: 46.50731100698336,
 qrstuvwxyz01: 160.4665572976518,
 tuvwxyz0123456: 37.353070900069945,
 uvwx: 90.1762407618508,
 uvwxyz: 65.33539975565049}

In [41]:
rewards.vault.pool

{2345: [(10000, 81),
  (-1250.0, 85),
  (-1093.75, 148),
  (-957.03125, 183),
  (10000, 194),
  (10000, 197),
  (-3337.40234375, 207),
  (10000, 217),
  (10000, 221),
  (-5420.22705078125, 237),
  (-4742.698669433594, 240),
  (10000, 260)],
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: [(10000, 58),
  (-1250.0, 134),
  (-1093.75, 135),
  (10000, 164),
  (-2207.03125, 190),
  (10000, 230),
  (-3181.15234375, 238),
  (10000, 241),
  (-4033.50830078125, 298)],
 HIJKL: [(10000, 161), (10000, 259), (10000, 267), (-3750.0, 276)],
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: [(10000, 88),
  (10000, 128),
  (10000, 136),
  (-3750.0, 147),
  (-3281.25, 149),
  (-2871.09375, 166),
  (10000, 187),
  (-3762.20703125, 227),
  (10000, 252)],
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: [(10000, 44),
  (10000, 62),
  (-2500.0, 211),
  (10000, 266),
  (10000, 297)],
 MN: [(10000, 9),
  (10000, 127),
  (-2500.0, 167),
  (-2187.5, 223),
  (-1914.0625, 225),
  (10000, 280),
  (10000, 299)],
 NOPQRSTUVWX: [(1000

In [42]:
rewards.stakes.pool

{2345: [(1000, 12, 101), (1000, 12, 106), (1000, 12, 175), (1000, 12, 284)],
 DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: [(1000, 12, 60),
  (1000, 12, 165),
  (1000, 12, 222),
  (1000, 12, 278),
  (1000, 12, 291)],
 HIJKL: [(1000, 12, 31),
  (1000, 12, 49),
  (1000, 12, 97),
  (1000, 12, 124),
  (1000, 12, 145),
  (1000, 12, 156),
  (1000, 12, 192),
  (1000, 12, 210)],
 JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: [(1000, 12, 10),
  (1000, 12, 96),
  (1000, 12, 234)],
 KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: [(1000, 12, 132),
  (1000, 12, 162)],
 MN: [(1000, 12, 108), (1000, 12, 131), (1000, 12, 151), (1000, 12, 202)],
 NOPQRSTUVWX: [(1000, 12, 99), (1000, 12, 123)],
 PQRSTUVWXYZabscdef: [(1000, 12, 72), (1000, 12, 77), (1000, 12, 215)],
 TUVWXYZabscdefghijklmnopqrstuvwxyz0123: [(1000, 12, 269)],
 VW: [(1000, 12, 82), (1000, 12, 126), (1000, 12, 273), (1000, 12, 294)],
 WXYZabs: [(1000, 12, 46),
  (1000, 12, 64),
  (1000, 12, 118),
  (1000, 12, 133),
  (1000, 12, 138),
  (1000, 12, 153),

In [43]:
action_history

[{KLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz: 'withdraw'},
 {VW: 'withdraw'},
 {uvwx: 'stake'},
 {qrstuvwxyz01: 'unstake'},
 {uvwx: 'unstake'},
 {uvwxyz: 'unstake'},
 {uvwxyz: 'withdraw'},
 {uvwxyz: 'unstake'},
 {MN: 'deposit'},
 {JKLMNOPQRSTUVWXYZabscdefghijklmnopqrstuvwxyz0: 'stake'},
 {opq: 'deposit'},
 {uvwxyz: 'deposit'},
 {uvwxyz: 'unstake'},
 {defghijklmnopqrstuvwxyz012: 'unstake'},
 {opq: 'deposit'},
 {DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: 'unstake'},
 {opq: 'unstake'},
 {uvwx: 'unstake'},
 {TUVWXYZabscdefghijklmnopqrstuvwxyz0123: 'deposit'},
 {2345: 'unstake'},
 {MN: 'unstake'},
 {WXYZabs: 'unstake'},
 {ghijklmnopqrstuvwxyz012345: 'stake'},
 {2345: 'withdraw'},
 {uvwxyz: 'withdraw'},
 {tuvwxyz0123456: 'unstake'},
 {ghijklmnopqrstuvwxyz012345: 'stake'},
 {defghijklmnopqrstuvwxyz012: 'unstake'},
 {gh: 'withdraw'},
 {defghijklmnopqrstuvwxyz012: 'deposit'},
 {HIJKL: 'stake'},
 {uvwx: 'unstake'},
 {DEFGHIJKLMNOPQRSTUVWXYZabscdefgh: 'unstake'},
 {PQRSTUVWXYZabscdef: 'withdraw'},
 {uvwx

In [44]:
rewards.reward_per_tick_history

[{'apys_1': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_2': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_3': {}, 'stake_pool': {}, 'vault_pool': {}},
 {'apys_4': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_5': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_6': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_7': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_8': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_9': {},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {}},
 {'apys_10': {MN: 7783.862000000001},
  'stake_pool': {uvwx: [(1000, 12, 3), (1000, 12, 200), (1000, 12, 282)]},
  'vault_pool': {MN: [(10000, 9),
    (10000, 127),
    